In [ ]:
#MS data pre-analysis by MS-DIAL 5.3. Only peak table as txt. format was needed.
#This script is specifically designed for carbon–carbon (C–C) backbone oligomers in water sample MS data.

import pandas as pd
import re
import numpy as np

peak_table = pd.read_table('WW.txt')
bk_table = pd.read_table('GCKB.txt')
output_path = 'WW_output.xlsx'  
Mode = "negative"

In [ ]:
######## Peaktable clean########
clean1 = 1
clean2 = 1

if clean1 == 1:
    # Ensure 'Isotope' column is treated as string
    peak_table['Isotope'] = peak_table['Isotope'].astype(str)

    # Filter rows where 'Isotope' is "0" or "1"
    peak_table = peak_table[peak_table['Isotope'].isin(["0", "1"])]

    # Reset index to keep the DataFrame tidy
    peak_table.reset_index(drop=True, inplace=True)

    # Define thresholds
    sn_threshold = 3          # Signal-to-noise (S/N) ratio threshold
    area_threshold = 10000    # Area threshold
    mz_threshold = 0.005      # m/z threshold for grouping
    rt_threshold = 0.2        # RT (Retention Time) threshold for grouping

    # Filter based on S/N and Area thresholds
    peak_table = peak_table.query('`S/N` >= @sn_threshold and `Area` >= @area_threshold')

    # Sort by 'Precursor m/z' for consistent grouping
    peak_table = peak_table.sort_values('Precursor m/z')

    # Initialize an empty list to store grouped rows
    groups = []

    # Group peaks based on m/z and RT thresholds, keeping the one with the maximum Area
    while not peak_table.empty:
        # Identify the group based on thresholds
        group_mask = (peak_table['Precursor m/z'].sub(peak_table.iloc[0]['Precursor m/z']).abs() <= mz_threshold) & \
                     (peak_table['RT (min)'].sub(peak_table.iloc[0]['RT (min)']).abs() <= rt_threshold)
        group = peak_table[group_mask]

        # Keep the row with the maximum Area within the group
        largest_area_row = group.loc[group['Area'].idxmax()]
        groups.append(largest_area_row)

        # Remove grouped rows from the DataFrame
        peak_table = peak_table[~group_mask]

    # Create the final DataFrame from grouped rows
    peak_table = pd.DataFrame(groups)

if clean2 == 1:    
    # Parameters for blank deduction
    proportion = 0.3  # Signal intensity proportion cutoff
    # Initialize an empty set to store indices of rows to remove
    to_remove = set()

    # Iterate over each row in the peak table
    for i, peak_row in peak_table.iterrows():
        for _, bk_row in bk_table.iterrows():
            # Calculate m/z difference
            mz_difference = abs(peak_row['Precursor m/z'] - bk_row['Precursor m/z'])
            if mz_difference <= mz_threshold:
                # Calculate RT difference
                rt_difference = abs(peak_row['RT (min)'] - bk_row['RT (min)'])
                if rt_difference <= rt_threshold:
                    # Calculate intensity proportion
                    proportion_cal = peak_row['Area'] / bk_row['Area']
                    if proportion_cal <= proportion:
                        to_remove.add(i)
                        break  # Stop checking further blanks for this peak

    # Remove rows marked for removal
    peak_table = peak_table.drop(index=list(to_remove)).reset_index(drop=True)

# Display the cleaned peak_table

peak_table

In [ ]:
####Exact mass calculation#########
columns_to_drop = [
    'Sharpness', 'Gaussian similarity', 'Ideal slope', 'Symmetry', 'Model masses', 
     'Isotope', 'Comment', 'Reference RT', 'Reference m/z', 'Formula', 
    'Ontology', 'InChIKey', 'SMILES', 'Annotation tag (VS1.0)', 'RT matched', 
    'm/z matched', 'MS/MS matched', 'RT similarity', 'm/z similarity', 
    'Simple dot product', 'Weighted dot product', 'Reverse dot product', 
    'Matched peaks count', 'Matched peaks percentage', 'Total score', 'S/N'
]

peak_table = peak_table .drop(columns=columns_to_drop)


def Get_Adduct_Mass(Mode, Adduct):
    Z = 0
    
    if Mode == "negative":
        if Adduct == "[M-H]-":
            Z = -1.0072766
        elif Adduct == "[M+H2O-H]-":
            Z = 17.0032881
        elif Adduct == "[M+Cl]-":
            Z = 33.9615761
        elif Adduct == "[M+FA-H]-":
            Z = 44.9982027
        elif Adduct == "[M+Hac-H]-":
            Z = 59.0138527
        elif Adduct == "[M-2H]2-":
            Z = -2.0145532
    
    if Mode == "positive":
        if Adduct == "[M+H]+":
            Z = 1.0072767
        elif Adduct == "[M+Na]+":
            Z = 22.9892213
        elif Adduct == "[M+K]+":
            Z = 38.9631585
        elif Adduct == "[M+H-H2O]+":
            Z = -17.0021912
        elif Adduct == "[M+ACN+H]+":
            Z = 42.0338258
        elif Adduct == "[M+CH3OH+H]+":
            Z = 33.0334914
        elif Adduct == "[M+NH4]+":
            Z = 19.0416508
        elif Adduct == "[M+2H]2+":
            Z = 2.0145533
    
    return Z

peak_table['Mass'] = 0.0

for index, row in peak_table.iterrows():
    try:
        Adduct = row['Adduct']
        
        # 提取电荷
        match = re.search(r".*\](.)", Adduct)
        if match:
            Charge = match.group(1)
        else:
            print(f"Adduct format invalid at index {index}: {Adduct}")
            continue
        
        if Charge == "+" or Charge == "-":
            Charge = 1.0
        else:
            Charge = float(Charge)

        Precursor = row['Precursor m/z']
        Z = Get_Adduct_Mass(Mode, Adduct)

        Mass = Precursor * Charge - Z
        peak_table.at[index, 'Mass'] = Mass
    except Exception as e:
        print(f"Error at index {index}: {e}")

In [ ]:
########Oligomer finder by suspect screening###########

ppm = 1e-6  # Global variable.

peak_table['DP'] = 0
peak_table['EG'] = None
peak_table['RU'] = None

Olig_df = pd.read_excel('Plastic oligomer suspect list.xlsx')

for index, row1 in peak_table.iterrows():
    for _, row2 in Olig_df.iterrows():

        n_estimate = round((row1['Mass'] - row2['Differ mass']) / row2['NL mass'])
        if n_estimate <= 0:  
            continue
        
        # Check the upper and lower ranges of the estimates (e.g. ±2 range)
        for n in range(n_estimate - 3, n_estimate + 3):
            if n <= 0:  
                continue

            calculated_mass = row1['Mass'] - row2['NL mass'] * n - row2['Differ mass']
            if abs(calculated_mass) < 10 * ppm * row1['Mass']:  
                peak_table.at[index, 'DP'] = n
                peak_table.at[index, 'RU'] = row2['Acronyms']
                peak_table.at[index, 'EG'] = row2['Differ formula']
                break  

In [ ]:
########Diagnostic NL check#############

minint = 0.01
mass_tolerance_NL = 0.005  # Da

# 初始化 peak_table
peak_table['MS2feature'] = 'NaN'  
peak_table.reset_index(drop=True, inplace=True)

for i in range(len(peak_table)):
    # for di-carboxylic end structure
    if peak_table["EG"][i] == "C2H2O4":  
        nl = 43.9898292
        precursor = float(peak_table["Precursor m/z"][i])

        ms2peak_table = str(peak_table["MSMS spectrum"][i]).split(";")
        if ms2peak_table != ['nan']:
            peak_table.at[i, 'MS2feature'] = 'N'
            ms2peak_table = pd.DataFrame(
                [x.split(" ") for x in ms2peak_table],
                columns=["mz", "intensity"]
            )
            ms2peak_table['mz'] = pd.to_numeric(ms2peak_table['mz'])
            ms2peak_table['intensity'] = pd.to_numeric(ms2peak_table['intensity'])
            ms2peak_table["Rintensity"] = ms2peak_table['intensity'] / max(ms2peak_table['intensity'])

            ms2peak_table = ms2peak_table[ms2peak_table['mz'] <= precursor + mass_tolerance_NL]

            matched_row_index = None

            for j in range(len(ms2peak_table)):
                if (
                    abs(precursor - ms2peak_table["mz"].iloc[j] - nl) < mass_tolerance_NL and
                    ms2peak_table["Rintensity"].iloc[j] >= minint
                ):
                    matched_row_index = j
                    break

            # If a matching row is found, mark NL as 'Y'
            if matched_row_index is not None:
                peak_table.at[i, 'NL'] = 'Y'

In [ ]:
###### RT prediction #############
peak_table['PredictedRT'] = 'NaN'  

peak_table['Row_Index'] = peak_table.index  
grouped = peak_table.groupby(['RU', 'EG'])['Row_Index'].apply(list).reset_index()

grouped.rename(columns={'Row_Index': 'Row_Indices'}, inplace=True)
# Iterate over each group
for i in range(len(grouped)):
    # Get the row indices for the current group
    group_indices = grouped.at[i, 'Row_Indices']
    
    # Extract DP and RT (min) values for the current group
    dp_values = peak_table.loc[group_indices, 'DP']
    rt_values = peak_table.loc[group_indices, 'RT (min)']
    
    # Find the minimum dp value where dp > 3
    dp_min = dp_values[dp_values > 3].min() if (dp_values > 3).any() else None

# Find the largest RT value corresponding to dp_min
    if dp_min is not None:
        rt_min = rt_values[dp_values == dp_min].max()  # Selects the largest RT for dp_min
    else:
        rt_min = None

    # Find the maximum dp value where dp <= 12
    dp_max = dp_values[dp_values <= 12].max() if (dp_values <= 12).any() else None

# Find the largest RT value corresponding to dp_max
    if dp_max is not None:
        rt_max = rt_values[dp_values == dp_max].max()
    else:
        rt_max = None
    
    # Check if dp_min and dp_max are valid and not equal
    if dp_min is not None and dp_max is not None and dp_min != dp_max:
        # Fit the equation RT = k * ln(DP) + b
        k = (rt_max - rt_min) / (np.log(dp_max) - np.log(dp_min))
        b = rt_min - k * np.log(dp_min)
        
        # Update PredictedRT for each peak in the group
        for j in group_indices:
            dp = peak_table.at[j, 'DP']
            if dp > 1:  # Ensure DP is greater than 1 for log computation
                predicted_rt = k * np.log(dp) + b
                peak_table.at[j, 'PredictedRT (min)'] = round(predicted_rt, 3)
    else:
        # If dp_min and dp_max are equal or invalid, use rt_min directly
        for j in group_indices:
            peak_table.at[j, 'PredictedRT'] = rt_min if rt_min is not None else np.nan
columns_to_drop = ['RT left (min)', 'RT right (min)', 'Estimated noise', 
                   'S/N.1', 'MS1 isotopes', 'MSMS spectrum', 'Row_Index']

# Drop the specified columns from peak_table
peak_table.drop(columns=columns_to_drop, inplace=True)


In [ ]:
peak_table.to_excel(output_path, index=False) 